In [1]:
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.0 MB/s eta 0:00:00


In [1]:
from elasticsearch import Elasticsearch
import json
import os
import pandas as pd

# Create the client instance
client = Elasticsearch("http://localhost:9200")

# Successful response!
client.info()

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85043/2156522912.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.info()


ObjectApiResponse({'name': 'e2c53b10e2ca', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Gz8IOq2-SbyAPmMlvOylhw', 'version': {'number': '7.17.9', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ef48222227ee6b9e70e502f0f0daa52435ee634d', 'build_date': '2023-01-31T05:34:43.305517834Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [2]:
# NOTE: THe following code must be run just once
# Get the documents from the directory
entries = os.listdir('../1-publications/json')
for i in entries:
    # Opening JSON file
    f = open('../1-publications/json/'+i)

    # returns JSON object as a dictionary
    data = json.load(f)
    
    # Closing file
    f.close()

    # Remove the unnecessary fields
    list = ['url','paperId','year','publicationDate','authors']
    for i in list:
        data.pop(i, None)
    client.index(
    index='prueba',
    document=data)

# Refresh the index
client.indices.refresh(index='prueba')

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85043/1647646017.py:18: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.index(


JSONDecodeError: Extra data: line 1 column 1519 (char 1518)

In [3]:
# Query for the keyword you want
val = input("Enter the keyword you are looking for: ")
result = client.search(
 index='prueba',
  query={
    'match': {'abstract': val}
  }
 )
a = result['hits']['hits']
df = pd.DataFrame(columns=['title', 'abstract','score'])
for i in a:
  df['title'] = [i['_source']['title']]
  df['abstract'] = [i['_source']['abstract']]
  df['score'] = [i['_score']]
df

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85043/4254616861.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = client.search(


,title,abstract,score
0,"Soluble ST2, a biomarker of fibrosis, is assoc...",Abstract Several diseases have a deleterious f...,0.092931


In [10]:
# DO NOT RUN THIS CODE IF YOU DON'T WANT TO DELETE THE INDEX!!!
client.options(ignore_status=[400,404]).indices.delete(index='prueba')

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_84623/986347348.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.options(ignore_status=[400,404]).indices.delete(index='prueba')


ObjectApiResponse({'acknowledged': True})